<h1 align="center"><font color="yellow">Zero-shot, Few-shot and Prompt chaining using Langchain</font></h1>

<font color="yellow">Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro</font>

# [O que é LangChain](https://blog.gopenai.com/zeroshot-fewshot-and-prompt-chaining-using-langchain-4259d700d67f)

<font color="orange">Langchain é uma estrutura para desenvolvimento de aplicativos usando `LLMs`. Ele funciona como um `wrapper` para conexão com diferentes LLMs e abstrai a comunicação real entre seu aplicativo e o modelo de linguagem. Também pode funcionar como um compositor/orquestrador que reúne informações de diferentes modelos (ou mesmo sites via `agente`) e as combina para formar o resultado final do modelo.</font>

# <font color="red">Zero-shot Prompting</font>

O `prompt zero-shot` é um tipo de tarefa de processamento de linguagem natural (`NLP`) em que um modelo recebe um prompt e espera-se que gere um texto relevante para o prompt, mesmo que o modelo nunca tenha visto o prompt antes. `Por exemplo`, se a solicitação for `“Conte-me uma piada sobre um casal”`, espera-se que o modelo gere uma piada sobre um casal, mesmo que o modelo nunca tenha visto tal piada antes.

In [34]:
import os
import openai
import tiktoken
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [35]:
def get_prompt_template():
    return """
        Quero que você desempenhe o papel de um bot de viagens e responda à \
        pergunta. Retorne a resposta em formato json, você deve fornecer apenas três opções \
        listadas em uma estrutura de array. A raiz do array deve ser nomeada como "opções". \
        Você mesmo deve preencher os valores e remover opções inviáveis ou não fatíveis do json.        
        
        question: {question}
        """


In [37]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import json

class ZeroShotUtility:
    def __init__(self, template, temperature=0.0):
        self.template = template
        self.temperature = temperature

    def __str__(self):
        return f"{self.template}"

    def print_travel_modes(self, question):
        prompt_template = ChatPromptTemplate.from_template(self.template)
        message = prompt_template.format_messages(question=question)

        llm = ChatOpenAI(temperature=self.temperature, 
                         openai_api_key=os.getenv("OPENAI_KEY"))
        response = llm(message)
        
        print(response.content)
    

In [38]:
prompt_template = get_prompt_template()

zeroshotutility = ZeroShotUtility(prompt_template)


question = "Como chegar à cidade de Lima-Perú saindo de Vitória-ES Brasil?"
zeroshotutility.print_travel_modes(question=question)


{
  "opções": [
    "Opção 1: Voar de Vitória-ES para Lima-Perú",
    "Opção 2: Pegar um voo de Vitória-ES para São Paulo-SP e depois voar de São Paulo-SP para Lima-Perú",
    "Opção 3: Pegar um voo de Vitória-ES para Rio de Janeiro-RJ e depois voar de Rio de Janeiro-RJ para Lima-Perú"
  ]
}


<font color="orange">`OBSERVAÇÃO:`

Tenha cuidado com uso de `zero-shot`, já que têm muitos casos em que a estrutura da answer pode ser alterada. Portanto, se você deseja ter a answer em um formato específico ou se deseja que sua answer seja mais padronizada, o `zero-shot` pode não funcionar o tempo todo.</font>

# <font color="red">Few-shot Prompting</font>

O `few-shot prompt` é um tipo de tarefa de processamento de linguagem natural (`NLP`) em que um modelo recebe um `prompt` e `alguns exemplos de texto` e espera-se que gere texto que seja relevante para o prompt e `semelhante aos exemplos`.

<font color="orange">A seguir, forneceremos alguns exemplos sobre o tipo de resultado que desejamos. Além disso, você verá um `prefixo` e um `sufixo` apenas para separar a questão real dos exemplos no prompt (e para seguir a assinatura do método da biblioteca `Langchain`.)</font>

In [76]:
class FewShot:

    @staticmethod
    def get_examples():
        examples = [ 
            { 
                "question" : "Como chegar ao aeroporto de Calcutá até o Zoológico de Calcutá?" , 
                "answer" : """ 
                opção 1: mode=bus, min_time_in_min=75, max_time_in_min=90, description=pegue o ônibus nº 37 do aeroporto. Ele o deixará 
                em Alipore. Faça uma caminhada de 5 minutos, 
                opção 2: mode= metro, min_time_in_min=40, max_time_in_min=60, description=pegue o metrô do aeroporto. Ele 
                o deixará em Joka. Faça uma caminhada de 10 minutos
                opção 3: mod=trem, min_time_in_min=45, max_time_in_min=60, description=pegar o trem local do aeroporto. Isso 
                o deixará em Nova Alipore. Faça uma caminhada de 10 minutos, 
                opção 4: mod=walk, min_time_in_min=400, max_time_in_min=500, description=take Faça uma caminhada de 500 minutos 
                """
             }, 
            { 
                "question" : "Como chegar ao aeroporto de Hyderabad até Hyderabad HiTech City ?" , 
                "answer" : """ 
                opção 1: mode=bus, min_time_in_min=60, max_time_in_min=75, description=pegue o ônibus nº 23 do aeroporto. Isso 
                o deixará na cidade de alta tecnologia.
                opção 2: mode=car, min_time_in_min=30, max_time_in_min=45, description=pegar transporte ola/uber/aeroporto. 
                Ele o deixará na 
                opção 3 da cidade HiTech: mode=motorbike, min_time_in_min=25, max_time_in_min=40. description=pegue uma moto.Ela 
                o deixará na cidade HiTech 
                """
             }, 
            { 
                "question" : "Como chegar ao aeroporto de Heathrow até Hounslow Central?" , 
                "answer" : """ 
                        opção 1: mode=tube, min_time_in_min= 10, max_time_in_min=15. description=pegue o metrô no aeroporto.

                        opção 2: mode=carro, min_time_in_min=25, max_time_in_min=35. description=pegue um carro alugado no 
                         aeroporto. Ele irá deixá-lo em Hounslow Central 
                        """
             } 
        ]
        return examples

    @staticmethod
    def get_example_template():
        template = """
        Pergunta: {question}
        Resposta: {answer}
        """
        example_variables = ["question", "answer"]
        return template, example_variables

    @staticmethod
    def get_prefix():
        return f"""
        Quero que você desempenhe o papel de um bot de viagens e responda à \
        pergunta. Retorne a resposta em formato json, você deve fornecer apenas seis opções \
        listadas em uma estrutura de array. A raiz do array deve ser nomeada como "opções". \
        Você mesmo deve preencher os valores e remover opções inviáveis ou não fatíveis do json.
        """

    @staticmethod
    def get_suffix():
        return """
                Pergunta: {question}
                """

In [77]:
import os
from langchain import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


class FewShotUtility:

    def __init__(self, examples, prefix, suffix, input_variables, example_template, example_variables):
        self.examples = examples
        self.prefix = prefix
        self.suffix = suffix
        self.input_variables = input_variables
        self.example_template = example_template
        self.example_variables = example_variables

    def get_prompt(self, question):
        prompt_template = FewShotPromptTemplate(
            examples=self.examples,
            example_prompt=self.get_prompt_template(),
            prefix=self.prefix,
            suffix=self.suffix,
            input_variables=self.input_variables
        )
        prompt = prompt_template.format(question=question)
        return prompt

    def get_prompt_template(self):
        example_prompt = PromptTemplate(
            input_variables=self.example_variables,
            template=self.example_template
        )
        return example_prompt

    @staticmethod
    def print_travel_modes(prompt):
        prompt_template = ChatPromptTemplate.from_template(prompt)
        message = prompt_template.format_messages()
        llm = ChatOpenAI(temperature=0, openai_api_key=os.getenv("OPENAI_KEY"))
        response = llm(message)
        print(response.content)


In [78]:
fewshot = FewShot() # Instancio a Classe

examples = fewshot.get_examples()
prefix = fewshot.get_prefix()
suffix = fewshot.get_suffix()
example_template, example_variables = fewshot.get_example_template()


fewShot = FewShotUtility(examples=examples,
                         prefix=prefix,
                         suffix=suffix,
                         input_variables=["question"],
                         example_template=example_template,
                         example_variables=example_variables
                        )


question = "Como chegar à cidade de Lima-Perú saindo de Vitória-ES Brasil?"
prompt = fewShot.get_prompt(question)
fewShot.print_travel_modes(prompt)


Resposta:  
                opção 1: mode=avião, min_time_in_min=360, max_time_in_min=420, description=pegue um voo direto de Vitória para Lima. 
                opção 2: mode=ônibus, min_time_in_min=1440, max_time_in_min=1800, description=pegue um ônibus de Vitória para Lima. 
                opção 3: mode=carro, min_time_in_min=4320, max_time_in_min=5040, description=dirija de Vitória para Lima. 
                opção 4: mode=trem, min_time_in_min=7200, max_time_in_min=8640, description=pegue um trem de Vitória para Lima. 
                opção 5: mode=barco, min_time_in_min=43200, max_time_in_min=50400, description=faça uma viagem de barco de Vitória para Lima. 
                opção 6: mode=bicicleta, min_time_in_min=129600, max_time_in_min=151200, description=faça uma viagem de bicicleta de Vitória para Lima.
